In [1]:

import pandas as pd
import numpy as np
import utils.utils as utils
import utils.dataloader as dataloader
from importlib import reload  
import matplotlib.pyplot as plt
import datetime as dt
import pickle
import talib

reload(dataloader)
dtref = dataloader.datetime_reference()

In [32]:
dataset=pd.read_csv('tpyrced_BTC_1min_data.csv').head(10000).iloc[::-1]

In [4]:
dataseteth=pd.read_csv('tpyrced_ETH_1min_data.csv')

In [33]:
print(dataset)

           start_datetime      open      high       low     close     volume
9999  2021-05-09 17:51:00  57381.76  57402.88  57381.76  57399.99   1.655996
9998  2021-05-09 17:52:00  57400.00  57427.62  57394.42  57414.70   3.509968
9997  2021-05-09 17:53:00  57421.60  57428.48  57377.55  57428.46   5.515905
9996  2021-05-09 17:54:00  57428.46  57431.69  57411.31  57431.69   1.609312
9995  2021-05-09 17:55:00  57431.69  57433.18  57406.71  57420.11   2.521779
...                   ...       ...       ...       ...       ...        ...
4     2021-05-16 16:26:00  47913.83  47919.65  47821.03  47830.47  13.111838
3     2021-05-16 16:27:00  47824.91  47861.03  47690.91  47705.30  17.864604
2     2021-05-16 16:28:00  47705.30  47780.04  47655.44  47676.69  13.341462
1     2021-05-16 16:29:00  47676.68  47684.01  47613.01  47628.05  33.276811
0     2021-05-16 16:30:00  47628.05  47639.95  47500.05  47508.04  18.830951

[10000 rows x 6 columns]


In [6]:
print(dataseteth)

             start_datetime     open     high      low    close       volume
0       2021-05-17 16:24:00  3302.32  3310.72  3296.70  3310.27  1485.519421
1       2021-05-17 16:23:00  3284.15  3303.23  3284.14  3302.32   886.965323
2       2021-05-17 16:22:00  3282.52  3289.17  3277.83  3284.29   519.962643
3       2021-05-17 16:21:00  3281.15  3283.05  3266.61  3282.69   590.651587
4       2021-05-17 16:20:00  3260.44  3282.31  3256.94  3280.89  2606.961591
...                     ...      ...      ...      ...      ...          ...
130580  2021-02-16 00:04:00  1777.05  1778.79  1775.83  1776.72   288.637475
130581  2021-02-16 00:03:00  1770.34  1777.19  1769.14  1777.05  1545.561250
130582  2021-02-16 00:02:00  1775.88  1778.34  1770.32  1770.34   739.311361
130583  2021-02-16 00:01:00  1781.75  1782.60  1775.07  1775.88   285.815289
130584  2021-02-16 00:00:00  1779.12  1782.97  1777.70  1781.75   860.639078

[130585 rows x 6 columns]


In [ ]:
sar_stra_v3(dataset, SAR_agent_v3, True, 0.00065)

In [ ]:
len(buy[buy == True])

In [15]:
def sar_stra_v3(time_bar, agent=SAR_agent, print_fig=False, use_fee=True):
    agent = agent(time_bar)
    
    buy, sell = agent.generate_signal_in_sar_stra(vol_ratio=1.25)
    
    def generate_sell_signal(df_input, buy_signal, sell_start=10, sell_end=60, profit_coef=1.005):
        df = df_input.copy()
        df['buy'] = buy_signal
        # init a sell-price vector
        df = df.apply(func=lambda x: x['close'] * profit_coef if x['buy'] else np.nan, axis=1)
        
        # generate a sell-price table with a drift
        df = pd.DataFrame(np.diag(df), columns=df.index)
        df = df.drop(df.index[df.sum() == 0], axis=0).replace(0, np.nan)

        if sell_end == -1:
            df = df.apply(func=lambda x: x.ffill(axis=0, inplace=False), axis=1)
        else:
            df = df.apply(func=lambda x: x.ffill(axis=0, inplace=False, limit=(sell_end - sell_start)), axis=1)
        df = df.shift(periods=sell_start, axis=1).fillna(0)
        
        def transform(row):
            p = ((row > 0) & (row <= df_input.close)).astype(int)
            if sum(p) > 0:
                return row[p == 1].index[0]
            elif sum(row != 0) > 0:
                return row[row != 0].index[-1]
            else:
                return row.index[-1]
     
        sell_loc = df.apply(func=transform, axis=1).values
        sell = pd.Series(data=0, index=df_input.index)
        sell.loc[sell_loc] = 1
        
        return sell
        
    sell = generate_sell_signal(time_bar, buy, sell_start=5, sell_end=-1, profit_coef=1.005)
        
    # position
    pos_rec = pd.Series(buy.astype(int))
    for i in range(1, len(pos_rec)):
        if (pos_rec[i-1] == 0) and sell[i]:
            pos_rec[i] = pos_rec[i-1]
        else:
            pos_rec[i] = pos_rec[i-1] + (1 if buy[i] and not sell[i] else -int(sell[i]))
            
    a = pos_rec

    real_buy = a.diff() > 0
    real_sell = a.diff() < 0
    
    # from trading
    if use_fee:
        fee = 0.0018
        c = (-(a.diff())*time_bar.close)
        c[c > 0] = c[c > 0] * (1 - fee)
        c[c < 0] = c[c < 0] * (1 + fee)
        # print(c[c > 0])
        # print(c[c < 0])
        c = c.cumsum()
    else:
        c = (-(a.diff())*time_bar.close).cumsum()
    
    b = (a)*time_bar.close # from holding stock

    value = (b+c)

    if print_fig:
        fig = agent.print_stat(real_buy, real_sell)
        ax = fig.add_subplot(5, 1, 4)
        ax.plot(value)
        
        ax = fig.add_subplot(5, 1, 5)
        ax.plot(a)
        
        fig.show()
        
        # plt.show()
        # plt.plot(time_bar.index, value)
        
        # print(value.min(), value.iloc[-1], value.min()/value.iloc[-1], buy.sum()/sell.sum())

    return value.iloc[-1]

In [18]:
def run_sar_backtest_v4(code_list, sar_stra, sar_agent, lot_df, fee):
    r = []
    
    date_list = None
    for code in code_list:
        try:
            data = dataloader.loadLOBDB(dtref, [int(code)], '201019', 60, level=2)
            LOB_data = utils.LOB_Data(*data)
            time_bars = LOB_data.resample_time_bar(60)
            if not date_list:
                date_list = time_bars.DateList
                
            tmp_r = []
            for j in range(60):
                if not time_bars[j].empty and time_bars[j].shape[0] > 100:
                    result = sar_stra(time_bars[j], sar_agent, lot=lot_df.loc[int(code)].lot, fee=fee)
                    # if result <= -5000:
                    #     sar_stra_v3(time_bars[j], SAR_agent_v1, True)
                    #     date_str = time_bars.DateList[j].strftime("%m%d")
                    #     img_name = f"{code}_{date_str}.jpg"
                    #     img_path = f"/home/zhsang/hkex/trading/sar_images/{img_name}"
                    #     plt.savefig(img_path, dpi='figure')
                    #     plt.close()
                    tmp_r.append(result if result else 0)
                else:
                    tmp_r.append(0)
            print(f"Code {code} earns {sum(tmp_r)}")
            r.append(tmp_r)
        except e:
            print(f"fail to process {code} with error {str(e)}")
            
    # r_np = np.array([np.array(rr) for rr in r if sum(rr) != 0])
    r_np = np.array([np.array(rr) for rr in r ])
    print_stats_from_result(r_np, date_list)
    
    return pd.DataFrame(r_np.T, columns=code_list, index=date_list)

In [ ]:
df_res_2 = run_sar_backtest_v4( sar_stra_v4, SAR_agent_v3, lot_df, 0.00065)

In [14]:
def sar_stra_v3(time_bar, agent=SAR_agent, print_fig=False, use_fee=True):
    agent = agent(time_bar)
    
    buy, sell = agent.generate_signal_in_sar_stra(vol_ratio=1.25)
    
    def generate_sell_signal(df_input, buy_signal, sell_start=10, sell_end=60, profit_coef=1.005):
        df = df_input.copy()
        df['buy'] = buy_signal
        # init a sell-price vector
        df = df.apply(func=lambda x: x['close'] * profit_coef if x['buy'] else np.nan, axis=1)
        
        # generate a sell-price table with a drift
        df = pd.DataFrame(np.diag(df), columns=df.index)
        df = df.drop(df.index[df.sum() == 0], axis=0).replace(0, np.nan)

        if sell_end == -1:
            df = df.apply(func=lambda x: x.ffill(axis=0, inplace=False), axis=1)
        else:
            df = df.apply(func=lambda x: x.ffill(axis=0, inplace=False, limit=(sell_end - sell_start)), axis=1)
        df = df.shift(periods=sell_start, axis=1).fillna(0)
        
        def transform(row):
            p = ((row > 0) & (row <= df_input.close)).astype(int)
            if sum(p) > 0:
                return row[p == 1].index[0]
            elif sum(row != 0) > 0:
                return row[row != 0].index[-1]
            else:
                return row.index[-1]
     
        sell_loc = df.apply(func=transform, axis=1).values
        sell = pd.Series(data=0, index=df_input.index)
        sell.loc[sell_loc] = 1
        
        return sell
        
    sell = generate_sell_signal(time_bar, buy, sell_start=5, sell_end=-1, profit_coef=1.005)
        
    # position
    pos_rec = pd.Series(buy.astype(int))
    for i in range(1, len(pos_rec)):
        if (pos_rec[i-1] == 0) and sell[i]:
            pos_rec[i] = pos_rec[i-1]
        else:
            pos_rec[i] = pos_rec[i-1] + (1 if buy[i] and not sell[i] else -int(sell[i]))
            
    a = pos_rec

    real_buy = a.diff() > 0
    real_sell = a.diff() < 0
    
    # from trading
    if use_fee:
        fee = 0.0018
        c = (-(a.diff())*time_bar.close)
        c[c > 0] = c[c > 0] * (1 - fee)
        c[c < 0] = c[c < 0] * (1 + fee)
        # print(c[c > 0])
        # print(c[c < 0])
        c = c.cumsum()
    else:
        c = (-(a.diff())*time_bar.close).cumsum()
    
    b = (a)*time_bar.close # from holding stock

    value = (b+c)

    if print_fig:
        fig = agent.print_stat(real_buy, real_sell)
        ax = fig.add_subplot(5, 1, 4)
        ax.plot(value)
        
        ax = fig.add_subplot(5, 1, 5)
        ax.plot(a)
        
        fig.show()
        
        # plt.show()
        # plt.plot(time_bar.index, value)
        
        # print(value.min(), value.iloc[-1], value.min()/value.iloc[-1], buy.sum()/sell.sum())

    return value.iloc[-1]

In [13]:
class SAR_agent_v3(SAR_agent_v2):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def generate_signal_in_sar_stra(self, roll_length=5, vol_ratio=1.5):
        buy, sell = super().generate_signal_in_sar_stra(roll_length, vol_ratio)
        
        tb = pd.Series(data=True, index=buy.index)
        tb[buy.ne(0).idxmax()] = True
        buy = tb
        
        sell = self.generate_sell_signal(self.time_bar, buy, sell_start=5, sell_end=-1, profit_coef=1.005)
        
        return buy, sell
        

In [10]:
class SAR_agent_v4(SAR_agent_v3):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def generate_signal_in_sar_stra(self, roll_length=5, vol_ratio=1.5):
        buy, sell = super().generate_signal_in_sar_stra(roll_length, vol_ratio)
        
        tb = pd.Series(data=False, index=buy.index)
        tb[buy.ne(0).idxmax()] = True
        buy = tb
        
        
        sell = self.generate_sell_signal(self.time_bar, buy, sell_start=5, sell_end=-1, profit_coef=1.005)
        
        return buy, sell

NameError: name 'SAR_agent_v3' is not defined

In [12]:
class SAR_agent_v2(SAR_agent_v1):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def generate_signal_in_sar_stra(self, roll_length=5, vol_ratio=1.5):
        buy, sell = super().generate_signal_in_sar_stra(roll_length, vol_ratio)
        
        sell = self.generate_sell_signal(self.time_bar, buy, sell_start=5, sell_end=-1, profit_coef=1.005)
        
        return buy, sell
    
    def generate_sell_signal(self, df_input, buy_signal, sell_start=10, sell_end=60, profit_coef=1.005):
        df = df_input.copy()
        df['buy'] = buy_signal
        # init a sell-price vector
        df = df.apply(func=lambda x: x['close'] * profit_coef if x['buy'] else np.nan, axis=1)
        
        # generate a sell-price table with a drift
        df = pd.DataFrame(np.diag(df), columns=df.index)
        df = df.drop(df.index[df.sum() == 0], axis=0).replace(0, np.nan)

        if sell_end == -1:
            df = df.apply(func=lambda x: x.ffill(axis=0, inplace=False), axis=1)
        else:
            df = df.apply(func=lambda x: x.ffill(axis=0, inplace=False, limit=(sell_end - sell_start)), axis=1)
        df = df.shift(periods=sell_start, axis=1).fillna(0)
        
        def transform(row):
            p = ((row > 0) & (row <= df_input.close)).astype(int)
            if sum(p) > 0:
                return row[p == 1].index[0]
            elif sum(row != 0) > 0:
                return row[row != 0].index[-1]
            else:
                return row.index[-1]
        
        sell_loc = df.apply(func=transform, axis=1).values
        sell_loc = sell_loc[sell_loc != df_input.index[-1]]
        if sell_loc.size > 0:
            sell = pd.Series(data=1, index=sell_loc).groupby(level=0).sum()
            sell = sell.reindex(df_input.index, fill_value=0)
        else:
            sell = pd.Series(data=0, index=df_input.index)
        
        return sell

In [10]:

class SAR_agent_v1(SAR_agent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def generate_signal_in_sar_stra(self, roll_length=5, vol_ratio=1.5):
        up = (self.sar > self.time_bar.close).astype(int)
        going_up = up.rolling(4, min_periods=1).sum() == 0
        
        down = (self.sar < self.time_bar.close).astype(int)
        down_diff = -down.diff()

        self.vol_avg = self.time_bar.volume.rolling(roll_length, min_periods=1).mean()
        # vol_exceed = (self.time_bar.volume > self.vol_avg * vol_ratio).astype(int).rolling(roll_length, min_periods=1).sum()
        # trading_cond = (vol_exceed > 0)

        macd_buy = (self.macd.macdhist > 0).astype(int).diff()
        macd_sell = (self.macd.macdhist < 0).astype(int).diff()
        nega_macd = (self.macd.macd < 0).astype(int)

        sar_sell = up.diff().rolling(roll_length, min_periods=1).sum()
        
        buy = (going_up == 1) & (macd_buy > 0) & (nega_macd > 0)
        sell = (down_diff == 1)

        return buy, sell

In [9]:
class SAR_agent:
    def __init__(self, time_bar, sar_acce=0.02, sar_max=0.2, macd_fast=12, macd_slow=26, macd_period=9):
        self.time_bar = time_bar
        self.sar = talib.abstract.SAR(time_bar, accelation=sar_acce, maximum=sar_max)
        self.macd = talib.abstract.MACD(time_bar, fastperiod=macd_fast, slowperiod=macd_slow, signalperiod=macd_period)
        
    def generate_signal_in_sar_stra(self, roll_length=5, vol_ratio=1.5):
        up = (self.sar > self.time_bar.close).astype(int)
        up_diff = -up.diff()

        self.vol_avg = self.time_bar.volume.rolling(roll_length, min_periods=1).mean()
        vol_exceed = (self.time_bar.volume > self.vol_avg * vol_ratio).astype(int).rolling(roll_length, min_periods=1).sum()

        macd_buy = (self.macd.macdhist > 0).astype(int).rolling(roll_length, min_periods=1).sum()
        macd_sell = (self.macd.macdhist < 0).astype(int).diff()
        nega_macd = (self.macd.macd < -10).astype(int).rolling(roll_length, min_periods=1).sum()

        trading_cond = (vol_exceed > 0)

        sar_sell = up.diff().rolling(roll_length, min_periods=1).sum()

        buy = (up_diff == 1) & (macd_buy > 0) & (macd_sell < 0) & trading_cond & (nega_macd > 0)
        sell = (sar_sell > 0) & (macd_sell > 0) & trading_cond

        return buy, sell
    
    def print_stat(self, real_buy, real_sell):
        fig, ax = plt.subplots(5, figsize=(20,10))

        ax[0].plot(self.time_bar.close)
        ax[0].plot(self.time_bar.close[real_buy],'v')
        ax[0].plot(self.time_bar.close[real_sell],'^')
        ax[0].plot(self.sar, '.', mfc='none')

        ax[1].plot(self.macd.macd, color='orange')
        ax[1].plot(self.macd.macdsignal, 'b')
        ax[1].hlines(y=0, xmin=self.time_bar.index[0], xmax=self.time_bar.index[-1], linestyles='--', color='brown')
        ax[1].plot(self.macd.macdhist, color='c', linestyle=':')
        ax[1].plot(self.macd.macdhist[real_buy],'v', color='orange')
        ax[1].plot(self.macd.macdhist[real_sell],'^', color='g')
        ax[1].legend(['DIF', 'DEA', 'MACD'])

        ax[2].plot(self.time_bar.volume, color='r', linestyle=':')
        ax[2].plot(self.vol_avg, 'b')
        ax[2].plot(self.time_bar.volume[real_buy],'v', color='orange')
        ax[2].plot(self.time_bar.volume[real_sell],'^', color='g')
        ax[2].legend(['volume', 'vol_5min'])

        ax[3].axis('off')
        ax[4].axis('off')
        # plt.show()
        
        return fig

In [8]:
def sar_stra_v1(time_bar, print_fig=False):
    agent = SAR_agent(time_bar)
    
    buy, sell = agent.generate_signal_in_sar_stra()

    # position
    a = ((buy).astype(int) - (sell)).cumsum()
    # a = -a
    
    pos_rec = pd.Series(buy.astype(int))
    for i in range(1, len(pos_rec)):
        if (pos_rec[i-1] == 0) and sell[i]:
            pos_rec[i] = pos_rec[i-1]
        else:
            pos_rec[i] = pos_rec[i-1] + (1 if buy[i] and not sell[i] else -int(sell[i]))
            
#   a =  pos_rec
#       real_buy = a.diff() < 0
#       real_sell = a.diff() > 0

    a = - pos_rec
    real_buy = a.diff() > 0
    real_sell = a.diff() < 0
    

    c = (-(a.diff())*time_bar.close).cumsum() # from trading
    b = (a)*time_bar.close # from holding stock

    value = (b+c)

    if print_fig:
        agent.print_stat(real_buy, real_sell)
        print(value.min(), value.iloc[-1], value.min()/value.iloc[-1], buy.sum()/sell.sum())

    return value.iloc[-1]

In [9]:
def sar_stra_v2(time_bar, agent=SAR_agent, print_fig=False):
    agent = agent(time_bar)
    
    buy, sell = agent.generate_signal_in_sar_stra(vol_ratio=1.25)

    # position
    a = ((buy).astype(int) - (sell)).cumsum()
    # a = -a
    
    pos_rec = pd.Series(buy.astype(int))
    for i in range(1, len(pos_rec)):
        if (pos_rec[i-1] == 0) and sell[i]:
            pos_rec[i] = pos_rec[i-1]
        else:
            pos_rec[i] = pos_rec[i-1] + (1 if buy[i] and not sell[i] else -int(sell[i]))
            
    a = pos_rec

    # a = - pos_rec
    real_buy = a.diff() > 0
    real_sell = a.diff() < 0

    c = (-(a.diff())*time_bar.close).cumsum() # from trading
    b = (a)*time_bar.close # from holding stock

    value = (b+c)

    if print_fig:
        fig = agent.print_stat(real_buy, real_sell)
        ax = fig.add_subplot(5, 1, 4)
        ax.plot(value)
        
        ax = fig.add_subplot(5, 1, 5)
        ax.plot(a)
        
        fig.show()
        
        # plt.show()
        # plt.plot(time_bar.index, value)
        
        print(value.min(), value.iloc[-1], value.min()/value.iloc[-1], buy.sum()/sell.sum())

    return value.iloc[-1]

In [10]:
sar_stra_v2(time_bars[33], True)

NameError: name 'time_bars' is not defined